In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy.stats import percentileofscore as score #The SciPy stats module

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2633079623695,
 'week52high': 167.4,
 'week52low': 118.32,
 'week52highSplitAdjustOnly': 170.4,
 'week52lowSplitAdjustOnly': 118.58,
 'week52change': 0.3572418522160702,
 'sharesOutstanding': 16414857605,
 'float': 0,
 'avg10Volume': 95857052,
 'avg30Volume': 75785963,
 'day200MovingAvg': 147.24,
 'day50MovingAvg': 157.43,
 'employees': 149420,
 'ttmEPS': 11.71,
 'ttmDividendRate': 0.86618006022429,
 'dividendYield': 0.005659264389292383,
 'nextDividendDate': '',
 'exDividendDate': '2021-10-25',
 'nextEarningsDate': '2022-01-24',
 'peRatio': 13.968293674259913,
 'beta': 1.471909012900277,
 'maxChangePercent': 59.68012594289546,
 'year5ChangePercent': 4.96813950770939,
 'year2ChangePercent': 1.38042993625487,
 'year1ChangePercent': 0.3674303966885425,
 'ytdChangePercent': 0.1968311951540709,
 'month6ChangePercent': 0.2647126346340512,
 'month3ChangePercent': 0.05919843083154602,
 'month1ChangePercent': 0.05603892177326533,
 'day30ChangePercent'

In [4]:
data['year1ChangePercent']

0.3674303966885425

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,156.83,0.348984,N/A
1,AAL,18.10,0.185262,N/A
2,AAP,229.60,0.53539,N/A
3,AAPL,160.66,0.354442,N/A
4,ABBV,116.87,0.171051,N/A
...,...,...,...,...
500,YUM,130.40,0.180841,N/A
501,ZBH,130.15,-0.16203,N/A
502,ZBRA,596.02,0.582683,N/A
503,ZION,66.69,0.679885,N/A


In [7]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,83.06,2.322541,N/A
1,DVN,43.70,2.054121,N/A
2,FTNT,341.45,1.688144,N/A
3,FANG,112.58,1.530305,N/A
4,MRO,15.90,1.502705,N/A
5,MCHP,80.45,1.438109,N/A
6,NVDA,317.50,1.394938,N/A
7,F,19.82,1.242821,N/A
8,NUE,118.92,1.147057,N/A
9,IT,334.82,1.144513,N/A


In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:10000000
10000000


In [9]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

C:\Users\Capol\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,83.06,2.322541,2360
1,DVN,43.70,2.054121,4486
2,FTNT,341.45,1.688144,574
3,FANG,112.58,1.530305,1741
4,MRO,15.90,1.502705,12331
5,MCHP,80.45,1.438109,2437
6,NVDA,317.50,1.394938,617
7,F,19.82,1.242821,9892
8,NUE,118.92,1.147057,1648
9,IT,334.82,1.144513,585


In [10]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

In [11]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

hqm_dataframe.replace([None], 0, inplace = True)

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100
# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0      0.645545
1      0.441584
2      0.823762
3      0.651485
4       0.39802
         ...   
500    0.435644
501    0.043564
502    0.865347
503    0.924752
504    0.663366
Name: One-Year Return Percentile, Length: 505, dtype: object
0       0.69901
1      0.033663
2      0.847525
3      0.889109
4      0.588119
         ...   
500    0.538614
501    0.051485
502    0.805941
503    0.722772
504     0.90297
Name: Six-Month Return Percentile, Length: 505, dtype: object
0      0.126733
1      0.128713
2      0.881188
3      0.728713
4      0.435644
         ...   
500    0.293069
501    0.069307
502    0.534653
503    0.855446
504     0.80198
Name: Three-Month Return Percentile, Length: 505, dtype: object
0      0.382178
1      0.136634
2      0.425743
3      0.813861
4      0.863366
         ...   
500    0.451485
501    0.033663
502    0.964356
503    0.669307
504    0.788119
Name: One-Month Return Percentile, Length: 505, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,154.53,N/A,0.351146,0.645545,0.109798,0.69901,-0.124248,0.126733,-0.021431,0.382178,N/A
1,AAL,18.06,N/A,0.191352,0.441584,-0.282650,0.033663,-0.123137,0.128713,-0.068733,0.136634,N/A
2,AAP,233.80,N/A,0.539638,0.823762,0.216165,0.847525,0.118416,0.881188,-0.015768,0.425743,N/A
3,AAPL,162.85,N/A,0.356548,0.651485,0.260743,0.889109,0.057761,0.728713,0.057218,0.813861,N/A
4,ABBV,121.29,N/A,0.169159,0.39802,0.061906,0.588119,-0.014522,0.435644,0.078313,0.863366,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,125.20,N/A,0.186263,0.435644,0.043447,0.538614,-0.055400,0.293069,-0.008915,0.451485,N/A
501,ZBH,129.27,N/A,-0.159610,0.043564,-0.249510,0.051485,-0.163343,0.069307,-0.144477,0.033663,N/A
502,ZBRA,598.08,N/A,0.579706,0.865347,0.188273,0.805941,0.004924,0.534653,0.140755,0.964356,N/A
503,ZION,65.78,N/A,0.682280,0.924752,0.127726,0.722772,0.103112,0.855446,0.027147,0.669307,N/A


In [12]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

In [13]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False, inplace = True)
hqm_dataframe.reset_index(drop = True, inplace = True)
hqm_dataframe = hqm_dataframe[:50]

In [14]:
portfolio_input()

Enter the value of your portfolio:10000000


In [15]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

C:\Users\Capol\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,NVDA,317.07,630,1.407803,0.988119,1.081823,0.99802,0.407036,0.984158,0.293842,0.99604,0.991584
1,F,20.16,9920,1.237748,0.986139,0.350590,0.956436,0.511063,0.99604,0.280059,0.994059,0.983168
2,MCHP,82.70,2418,1.499525,0.990099,1.111751,1.0,1.056884,1.0,0.114311,0.940594,0.982673
3,AMD,155.35,1287,0.781777,0.954455,0.984089,0.99604,0.407135,0.986139,0.274709,0.990099,0.981683
4,ANET,125.92,1588,0.855400,0.964356,0.491638,0.980198,0.346676,0.978218,0.252027,0.984158,0.976733
5,XLNX,234.09,854,0.650216,0.914851,0.838876,0.994059,0.441687,0.994059,0.251276,0.982178,0.971287
6,DVN,43.60,4587,2.042119,0.99802,0.652019,0.990099,0.427359,0.992079,0.079897,0.869307,0.962376
7,LB,79.97,2500,2.309572,1.0,0.838556,0.992079,0.225686,0.964356,0.080541,0.877228,0.958416
8,ALB,273.80,730,0.990382,0.976238,0.605865,0.986139,0.131200,0.893069,0.159006,0.968317,0.955941
9,SPG,161.78,1236,1.014132,0.978218,0.290880,0.920792,0.223784,0.962376,0.120428,0.952475,0.953465


In [16]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

In [17]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [18]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

In [19]:
writer.save()